# Problem Set 3 Part A: Predicting House Prices in Singapore

**Release Date:** 7 October 2024 1800H

**Due Date:** 19 October 2024 2359H

## Overview

We have learned how to solve a regression problem using linear regression in class.
In Part A of this problem set, we will apply our knowledge to solve a real-world problem. More
specifically, we will develop linear regression and polynomial regression models to predict
house prices in Singapore.

**Required Files**:
* ps3.ipynb
* housing_data.csv

**Plagiarism Policy**: Please refer to our [Course Policies](https://canvas.nus.edu.sg/courses/62323/pages/course-policies).

**IMPORTANT**: While it is possible to write and run Python code directly in Jupyter notebook, we recommend that you do this Problem set with an IDE using the .py file provided. An IDE will make debugging significantly easier.

**Post-Problem Set Survey**:
Your feedback is important to us! After completing Problem Set 3, please take a moment to share your thoughts by filling out this [survey](https://coursemology.org/courses/2851/surveys/2410).

## Orientation to files

**ps3.ipynb**:
The template for all your tasks is provided in this file. Some test cases have
been provided for you to check the output of your algorithm against the expected result. The tests are **not** comprehensive, and you are
encouraged to write your own tests to check for correctness.

**housing.csv**:
There are 90 housing data points. Each data point consists of 3 features:
* **floor_area_sqm** - size of the house in square meters
* **bedrooms** - number of bedrooms
* **schools** - number of primary schools within a 1km radius

Our target value is the **asking_price**, which is the price of the housing unit.

### IMPORTANT

Similar to PS0, your implementation in the following tasks **should not
involve any iteration, including `map` and `filter`, or recursion**. Instead, please work
with the operations available in NumPy<sup>&#x2020;</sup>. Solutions that violate this will be penalised.

There is, however, an exception for **Tasks 2.4 and 3.4**. In the pseudo-code for the
algorithm required, there is an explicit for loop for task 2.4 and a while loop for task 3.4. Hence, **only for these tasks**, you
may use a **single for/while loop** to iterate for the number of epochs required.

<sup>&#x2020;</sup> You are allowed to use any mathematical functions, but this **does not mean that you are allowed to
use *any* NumPy function** (there are NumPy functions that aren’t mathematical functions). For example,
`np.vectorize` is not allowed since it is iterative. If you are in doubt about which functions are allowed, please
ask in the forum (:

In [27]:
# Inital imports and setup

import os
import numpy as np

###################
# Helper function #
###################
def load_data(filepath):
    '''
    Load in the given csv filepath as a numpy array

    Parameters
    ----------
    filepath (string) : path to csv file

    Returns
    -------
        X, y (np.ndarray, np.ndarray) : (m, num_features), (m,) numpy matrices
    '''
    *X, y = np.genfromtxt(
        filepath,
        delimiter=',',
        skip_header=True,
        unpack=True,
    ) # default dtype: float
    X = np.array(X, dtype=float).T # cast features to int type
    return X, y.reshape((-1, 1))

data_filepath = 'housing_data.csv'
X, y = load_data(data_filepath)

## Part 1: Defining cost functions

We need to define cost functions before creating a linear regression model to calculate
the error between our prediction and the true values. We will define two cost functions:
Mean Squared Error (MSE) and Mean Absolute Error (MAE).

### Task 1.1: Mean Squared Error (MSE)

Write the function `mean_squared_error(y_true, y_pred)` that returns a number representing the mean squared error of the predictions.

The formula for Mean Squared Error is as follows:
$$ MSE(\boldsymbol{y}, \boldsymbol{\hat{y}}) = \frac{1}{2m} \sum_{i=1}^{m}(\hat{y}_i - y_i)^2 $$

where $\boldsymbol{y}$ is the vector with actual values, $\boldsymbol{\hat{y}}$ is the prediction vector, and $m$ is the number of samples in the
training data.

**Remark**: The formula here follows the lecture slides for consistency. In definitions and implementations elsewhere, the denominator is usually just $m$ instead of $2m$.

**Hint**: Consider using `np.square` or `np.power`.


In [28]:
def mean_squared_error(y_true, y_pred):
    '''
    Calculate mean squared error between y_pred and y_true.

    Parameters
    ----------
    y_true (np.ndarray) : (N, 1) numpy matrix consists of true values
    y_pred (np.ndarray)   : (N, 1) numpy matrix consists of predictions
    
    Returns
    -------
        The mean squared error value.
    '''
    
    """ YOUR CODE HERE """
    mse = np.mean(np.square(y_true - y_pred))
    return mse
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [29]:
# Test cases
y_true, y_pred = np.array([[3], [5]]), np.array([[12], [15]])

assert mean_squared_error(y_true, y_pred) in [45.25, 90.5]

### Task 1.2: Mean Absolute Error (MAE)

Write the function `mean_absolute_error(y_true, y_pred)` that returns a number representing the mean absolute error of the predictions.

The formula for Mean Absolute Error is as follows:
$$ MSE(\boldsymbol{y}, \boldsymbol{\hat{y}}) = \frac{1}{m} \sum_{i=1}^{m}|\hat{y}_i - y_i| $$

where $\boldsymbol{y}$ is the vector with actual values, $\boldsymbol{\hat{y}}$ is the prediction vector, and $m$ is the number of samples in the
training data.

**Hint**: Consider using `np.abs`.


In [30]:
def mean_absolute_error(y_true, y_pred):
    '''
    Calculate mean absolute error between y_pred and y_true.

    Parameters
    ----------
    y_true (np.ndarray) : (m, 1) numpy matrix consists of true values
    y_pred (np.ndarray)   : (m, 1) numpy matrix consists of predictions
    
    Returns
    -------
        The mean absolute error value.
    '''
  
    """ YOUR CODE HERE """
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [31]:
# Test cases
y_true, y_pred = np.array([[3], [5]]), np.array([[12], [15]])

assert mean_absolute_error(y_true, y_pred) == 9.5

NotImplementedError: 

## Part 2: Linear Regression

Now we’re ready to create our own linear regression model. We will try to approximate a linear function, which can be written as follows:

$$ y = w_0 + w_1 x_1 + w_2 x_2 + \dots + w_n x_n $$

where $y$ is the target value, $x_1, x_2, \dots, x_n$ are feature values, and $w_0, w_1, \dots, w_n$ are parameters. $w_0$ is meant to represent the bias term, while $w_1, \dots, w_n$ are the feature weights.

**Bias term**

The bias term ($w_0$) is useful in capturing an inherent offset of the target values from the origin, i.e. they have some non-zero "default" or "starting" value. The bias term accounts for this default value in our model. Without a bias term (or bias = 0), our regression lines will pass through the origin, which might not be appropriate for the data in question.

Consider the scatter plot below. The blue line is the best fitting line without a bias term, while the red line includes a non-zero bias. Since the blue line starts at the origin, it is unable to capture the offset of the points. In contrast, the red line starts higher (at around 5), and hence is better able to approximate the data.

&nbsp;
<figure>
<img src="./images/bias_scatter.png" alt="bias vs no bias" width="40%">
<figcaption style="text-align:center">Figure 0: Example of models with bias vs without bias.</figcaption>
</figure>

Usually, we have to explicitly add a bias term into our data when building our models. In the following tasks, you'll explore how to do so and how this choice can affect the accuracy of your models.

### Task 2.1: Adding a bias column

In the lecture, we learned that adding a bias column allows our linear model to be more
flexible. Write the function `add_bias_column(X)` that takes a NumPy matrix `X` and returns
a new matrix with an additional column. The additional column should have all of its
elements set to 1 and is located at the first column of the matrix.

<figure>
<img src="./images/add_bias.jpeg" alt="adding bias" width="50%">
<figcaption style="text-align:center">Figure 1: Example of a matrix before and after adding a bias column.</figcaption>
</figure>

**Note**: Your function should work for all kinds of matrix shapes.

**Hint**: Consider using `np.hstack` to add the bias column.


In [ ]:
def add_bias_column(X):
    '''
    Create a bias column and combine it with X.

    Parameters
    ----------
    X : (N, d) numpy matrix representing a feature matrix
    
    Returns
    -------
        new_X (np.ndarray):
            A (N, d + 1) numpy matrix with the first column consisting of all 1s
    '''
  
    """ YOUR CODE HERE """
    n = np.shape(X)[0]
    new_col = np.ones((n, 1))
    return np.column_stack((new_col, X))
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [ ]:
# Test cases
without_bias = np.array([[1, 2], [3, 4]])
expected = np.array([[1, 1, 2], [1, 3, 4]])

assert np.array_equal(add_bias_column(without_bias), expected)

### Task 2.2: Get best fitting bias and weights

Write the function `get_bias_and_weight(X, y, include_bias)` that returns $w_0$ (bias) and
$w_1, w_2, \dots, w_n$ (weights) that will lead to best fitting line.

The `include_bias` argument is used to specify if the model includes a bias term, i.e. has a non-zero bias term. Hence, the function should return $w_0 = 0$ if it is set to `false`. The function should return $w_1, \dots, w_n$ as a NumPy matrix with shape $(n, 1)$, where $n$ is the number of features (excluding the bias column).

We can use the normal equation to get $w_0, w_1, \dots, w_n$. The normal equation is as
follows:

$$ \begin{pmatrix} w_0 \\ w_1 \\ \vdots \\ w_n \end{pmatrix} = (X^TX)^{-1}X^T \boldsymbol{y} $$

where $X$ is the (augmented for bias) feature matrix and $\boldsymbol{y}$ is the vector of target values.

**Note**: You can use the `add_bias_column` function for this task. (You do not need to re-define the function in Coursemology. However, you are free to do so if you explicitly want to use your own implementation of the function for this task.)

**Hint**: Consider using `numpy.linalg.inv` for the matrix inverse.


In [ ]:
def get_bias_and_weight(X, y, include_bias = True):
    '''
    Calculate bias and weights that give the best fitting line.

    Parameters
    ----------
    X (np.ndarray) : (m, n) numpy matrix representing feature matrix
    y (np.ndarray) : (m, 1) numpy matrix representing target values
    include_bias (boolean) : Specify whether the model should include a bias term
    
    Returns
    -------
        bias (float):
            If include_bias = True, return the bias constant. Else,
            return 0
        weights (np.ndarray):
            A (n, 1) numpy matrix representing the weight constant(s).
    '''
  
    """ YOUR CODE HERE """
    Z = np.round(add_bias_column(X))
    Y = np.round(y)

    if include_bias:
        w = np.linalg.inv(np.transpose(Z) @ Z) @ np.transpose(Z) @ Y
        return (w[0, 0], w[1: ])
    else :
        w = np.linalg.inv(np.transpose(X) @ X) @ np.transpose(X) @ Y
        return (0, w)
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [ ]:
# Test cases
public_X, public_y = np.array([[1, 3], [2, 3], [3, 4]]), np.arange(4, 7).reshape((-1, 1))

test_1 = (round(get_bias_and_weight(public_X, public_y)[0], 5) == 3)
test_2 = np.array_equal(np.round(get_bias_and_weight(public_X, public_y)[1], 1), np.array([[1.0], [0.0]]))
test_3 = np.array_equal(np.round(get_bias_and_weight(public_X, public_y, False)[1], 2), np.round(np.array([[0.49], [1.20]]), 2))

assert test_1 and test_2 and test_3

[[1. 1. 3.]
 [1. 2. 3.]
 [1. 3. 4.]]
(2.9999999999999645, array([[1.00000000e+00],
       [1.77635684e-14]]))
[[1. 1. 3.]
 [1. 2. 3.]
 [1. 3. 4.]]
(2.9999999999999645, array([[1.00000000e+00],
       [1.77635684e-14]]))
[[1. 1. 3.]
 [1. 2. 3.]
 [1. 3. 4.]]
(0, array([[0.48571429],
       [1.2       ]]))


### Task 2.3: Get the prediction line

Write the function `get_prediction_linear_regression(X, y, include_bias)` that returns `y_pred`,
a vector of predicted values for the training data.

**Note**: You can use the `get_bias_and_weight` function for this task. (You do not need to re-define the function in Coursemology. However, you are free to do so if you explicitly want to use your own implementation of the function for this task.)


In [ ]:
def get_prediction_linear_regression(X, y, include_bias = True):
    '''
    Calculate the best fitting line.

    Parameters
    ----------
    X (np.ndarray) : (m, n) numpy matrix representing feature matrix
    y (np.ndarray) : (m, 1) numpy matrix representing target values
    include_bias (boolean) : Specify whether the model should include a bias term

    Returns
    -------
        y_pred (np.ndarray):
            A (m, 1) numpy matrix representing prediction values.
    '''
    
    """ YOUR CODE HERE """
    bias, w = get_bias_and_weight(X, y, include_bias)
    if include_bias:
        X = add_bias_column(X)
        w = np.vstack(([bias], w))
    return X @ w
    """ YOUR CODE END HERE """

In [ ]:
# Test cases
test_X, test_y = np.array([[1, 3], [2, 3], [3, 4]]), np.arange(4, 7).reshape((-1, 1))

assert round(mean_squared_error(test_y, get_prediction_linear_regression(test_X, test_y)), 5) == 0

To check your work, create a linear regression model with **floor_area_sqm** as the only
feature and **asking_price** as the target value. Plot your prediction line using the code
snippet below. It should look similar to Figure 2.

&nbsp;
<figure>
<img src="./images/linear_reg.png" alt="regression plot" width="50%">
<figcaption style="text-align:center">Figure 2: Example of linear regression using <b>floor_area_sqm</b> as feature.</figcaption>
</figure>

In [ ]:
import matplotlib.pyplot as plt

area = X[:, 0].reshape((-1, 1))
predicted = get_prediction_linear_regression(area, y)
plt.scatter(area, y)
plt.plot(area, predicted, color = 'r')
plt.xlabel("Size in square meter")
plt.ylabel("Price in SGD")
plt.show()

## Gradient Descent

We will now learn to use gradient descent to approximate $\boldsymbol{w} = w_0, w_1, \dots, w_n$.

*Gradient descent*<sup>&#x2020;</sup> is an algorithm that minimizes the cost function by iteratively trying to
find the best parameters. In linear regression, we will try to minimize the Mean Squared
Error. The outline of the algorithm is as follows:
    
* Start with some $\boldsymbol{w} = (w_0, \dots, w_n)$
* Keep changing $w_0,\dots, w_n$ to minimize $J(\boldsymbol{w})$, where $J$ is our cost function

In this problem set, we will initially set $w_0, w_1, \dots, w_n$ to all be 0s. Then, we will set a
learning rate $\alpha$ that will affect the rate of change of $w_0, \dots, w_n$. Lastly, we will set
$N$ to specify the number of epochs of gradient descent we want to run.

The pseudo-code of Gradient Descent for linear regression is defined in Algorithm 1.

**Note**: In the following gradient descent-related tasks, calculate the value of the loss function *after* updating the bias and weights.

<sup>&#x2020;</sup> *The Gradient Descent algorithm is not limited to the linear regression model – it is a general optimisation technique and is also used in many other machine learning models such as Neural Networks.*

<div style="display: flex; justify-content: center; align-items: center;">
    <img src="./images/mse_plot.png" alt="gradient descent" style="width: 45%; margin-right: 10px;">
    <img src="./images/grad_desc_algorithm.png" alt="gradient descent" style="width: 45%;">
</div>
<figcaption style="text-align:left; margin-top: 10px;">Figure 3: Gradient descent tries to find parameters that lead to the lowest MSE.</figcaption>

For MSE, The partial derivative $\frac{\partial J(\boldsymbol w)}{\partial w_i}$ with $m$ training samples can be derived as: 
$$
\frac{\partial J(\boldsymbol{w})}{\partial w_i} = \frac 1m\sum^m_{j=1}(h_w(x^{(j)})-y^{(j)})\cdot x_i^{(j)} 
$$
where $h_w$ is our hypothesis.

### Task 2.4: Gradient Descent on multiple features

Write the function `gradient_descent_multi_variable(X, y, lr, number_of_epochs)` that returns:

* $w_0$ - a number representing the bias constant
* $w_1, w_2, \dots, w_n$ - $(n,1)$ NumPy matrix, where each element denotes the weight constant of a certain feature
* $loss$ - a list that contains the MSE scores calculated during the gradient descent process.

**Note**: You can use the `mean_squared_error` function for this task. (You do not need to re-define the function in Coursemology. However, you are free to do so if you explicitly want to use your own implementation of the function for this task.)


In [ ]:
def gradient_descent_multi_variable(X, y, lr = 1e-5, number_of_epochs = 250):
    '''
    Approximate bias and weight that gave the best fitting line.

    Parameters
    ----------
    X (np.ndarray) : (m, n) numpy matrix representing feature matrix
    y (np.ndarray) : (m, 1) numpy matrix representing target values
    lr (float) : Learning rate
    number_of_epochs (int) : Number of gradient descent epochs
    
    Returns
    -------
        bias (float):
            The bias constant
        weights (np.ndarray):
            A (n, 1) numpy matrix that specifies the weight constants.
        loss (list):
            A list where the i-th element denotes the MSE score at i-th epoch.
    '''
    # Do not change
    bias = 0
    weights = np.full((X.shape[1], 1), 0).astype(float)
    loss = []
    
    N = X.shape[0]  
    pred = X @ weights + bias
    for e in range(number_of_epochs):
        pred = X @ weights + bias
        g_w = (1 / N) * (X.T @ (pred - y))
        g_b = (1 / N) * np.sum(pred - y)
        bias -= lr * g_b
        weights -= lr * g_w
        loss.append(mean_squared_error(y, pred))
    return bias, weights, loss  

In [ ]:
# Test cases
_, _, loss = gradient_descent_multi_variable(X, y, lr = 1e-5, number_of_epochs = 250)
loss_initial = loss[0]
loss_final = loss[-1]

assert loss_initial > loss_final

### Task 2.5: Which algorithm should we use for Linear Regression?

Compare the pros and cons of using normal equation and gradient descent for linear regression. Specifically:

- Compare the speed of the two algorithms on data with many features. 
- Compare the quality of the solutions obtained by the two algorithms. (i.e. how close to the optimal solution are the solutions obtained by the algorithms)
- Compare whether feature scaling is necessary for each algorithm to perform well.

Finally, select the algorithm you think is more suitable for this problemset and explain why you chose it.


## Part 3: Polynomial Regression

In real-world data, a straight line might not fit the data perfectly. Consider the relation between **schools** and **asking_price**.

&nbsp;
<figure>
<img src="./images/school_price_rel.png" alt="school price relation" width="50%">
<figcaption style="text-align:center">Figure 4: Schools - Price Relationship.</figcaption>
</figure>

Houses with 0 schools nearby tend to be cheaper than houses with 1 school nearby. However, as the number of schools increases, the prices decrease. If we try a linear regression on the data, we obtain the following:

&nbsp;
<figure>
<img src="./images/school_price_rel_linearfit.png" alt="school price relation linear fit" width="50%">
<figcaption style="text-align:center">Figure 4.1: Schools - Price Relationship With Linear Fit.</figcaption>
</figure>

Notice how we lose the detail that houses with 0 schools are actually cheaper than houses with 1 school nearby. A polynomial function can better capture this relationship:

&nbsp;
<figure>
<img src="./images/school_price_rel_cubicfit.png" alt="school price relation cubic fit" width="50%">
<figcaption style="text-align:center">Figure 4.2: Schools - Price Relationship With Cubic Fit.</figcaption>
</figure>

A polynomial function is written as follows:

$$ y = w_0 + w_1 x + w_2 x^2 + ... + w_n x^n $$

where $y$ is the target value, $x$ is a (*single*) feature value, and $n$ is the degree of the polynomial. $w_0$ is the bias term and $w_1, \dots, w_n$ are the feature weights. 

Notice how if we set $x_1 = x, x_2 = x^2, \dots, x_n = x^n$, then the polynomial function is simply linear regression with $n$ features:

$$ y = w_0 + w_1 x_1 + w_2 x_2 + ... + w_n x_n $$

### Task 3.1 : Create Polynomial Matrix

Write the function `create_polynomial_matrix(X, power)` that takes a $(m, 1)$-matrix and an
integer, and returns a polynomial matrix with shape $(m, power)$.

$$
\left[\begin{array}{cc} 
1\\
2\\
3
\end{array}\right]
\xrightarrow[]{\text{create\_polynomial\_matrix(3)}}
\left[\begin{array}{cc} 
1 & 1^2 & 1^3\\ 
2 & 2^2 & 2^3\\
3 & 3^2 & 3^3
\end{array}\right]
\rightarrow
\left[\begin{array}{cc} 
1 & 1 & 1\\ 
2 & 4 & 8\\
3 & 9 & 27
\end{array}\right]
$$

**Hint**: Consider using `np.tile`/`np.repeat` together with `np.cumprod`/`np.power`.


In [ ]:
def create_polynomial_matrix(X, power = 2):
    '''
    Create a polynomial matrix.
    
    Parameters
    ----------
    X: (m, 1) numpy matrix

    Returns
    -------
        A (m, power) numpy matrix where the i-th column denotes
            X raised to the power of i.
    '''
    """ YOUR CODE HERE """
    return np.cumprod(np.tile(X, power), axis=1)
    
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [ ]:
# Test cases
vector = np.array([[1], [2], [3]])
poly_matrix = np.array([[1, 1, 1], [2, 4, 8], [3, 9, 27]])

assert np.array_equal(create_polynomial_matrix(vector, 3), poly_matrix)

### Task 3.2: Get the prediction line

Write the function `get_prediction_poly_regression(X, y, power, include_bias)` that returns
`y_pred`, a vector of predicted values for the training data.

**Note**: You can use the functions `create_polynomial_matrix` and `get_prediction_linear_regression` from before for this task. (You do not need to re-define the functions in Coursemology. However, you are free to do so if you explicitly want to use your own implementation of the functions for this task.)


In [32]:
def get_prediction_poly_regression(X, y, power = 2, include_bias = True):
    '''
    Calculate the best polynomial line.

    Parameters
    ----------
    X (np.ndarray) : (m, 1) numpy matrix representing feature matrix
    y (np.ndarray) : (m, 1) numpy matrix representing target values
    power (int) : Specify the degree of the polynomial
    include_bias (boolean) : Specify whether the model should include a bias term

    Returns
    -------
        A (m, 1) numpy matrix representing prediction values.
    '''
    """ YOUR CODE HERE """
    x = create_polynomial_matrix(X, power)
    return get_prediction_linear_regression(x, y, include_bias)
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [33]:
# Test cases
test_X, test_y = np.arange(3).reshape((-1, 1)), np.arange(4, 7).reshape((-1, 1))
pred_y = get_prediction_poly_regression(test_X, test_y, 2)

assert round(mean_squared_error(test_y, pred_y), 5) == 0

NameError: name 'get_prediction_linear_regression' is not defined

To check your work, create a polynomial regression model, using `power = 3` and `include_bias = True`, with **schools** as the only feature and **asking_price** as the target value. Plot your prediction line using the code snippet below. It should look similar to Figure 5.

&nbsp;
<figure>
<img src="./images/poly_reg.png" alt="polynomial regression" width="50%">
    <figcaption style="text-align:center">Figure 5: Example of polynomial regression using <b>schools</b> as feature.</figcaption>
</figure>

In [ ]:
import matplotlib.pyplot as plt

schools = X[:, 2].reshape((-1, 1))
predicted = get_prediction_poly_regression(schools, y, 3)
plt.scatter(schools, y)
plt.scatter(schools, predicted, color = 'r', s = 100)
plt.xlabel("Number of schools within 1km")
plt.ylabel("Price in SGD")
plt.show()

### Task 3.3: Feature Scaling

As we create a higher degree polynomial matrix, each column will have a larger scale
than the previous one. This can lead to poor performance for gradient descent. Here
is where feature scaling plays an important role. Write the function `feature_scaling(X)`
that takes a NumPy matrix `X` and returns a mean-normalized matrix.

**Note**: The normalization occurs on the column level (i-th column is normalized by the
mean and standard deviation of the i-th column). That is,

$$
\text{If} \quad \boldsymbol{v} = \begin{pmatrix} v_1 \\ v_2 \\ \vdots \\ v_k \end{pmatrix} \\
\boldsymbol{v}_{norm} = \frac{\boldsymbol{v} - \boldsymbol{\hat{v}}}{\sigma_{v}}
$$

where $\boldsymbol{v}$ is a vector of $k$ elements, $\boldsymbol{\hat{v}}$ is its mean, and $\sigma_{v}$ is its standard deviation.

$$
\left[\begin{array}{cc} 
1 & 133\\
4 & 700\\
5 & 133\\
8 & 700
\end{array}\right]
\xrightarrow[]{\text{feature\_scaling}}
\left[\begin{array}{cc} 
-1.4 & -1\\
-0.2 & 1\\
0.2 & -1\\
1.4 & 1
\end{array}\right]
$$ 

Focusing on the first feature:

$$
\boldsymbol{v} = \left(\begin{array}{cc}
1\\
4\\
5\\
8
\end{array}\right), \boldsymbol{\hat{v}} = 4.5, \sigma_{v} = 2.5$$

$$\boldsymbol{v}_{norm} = \frac{\boldsymbol{v} - 4.5}{2.5} = \left(\begin{array}{cc}
-1.4\\
-0.2\\
0.2\\
1.4
\end{array}\right)
$$

**Hint**: Consider using `np.mean()` and `np.std()`.


In [34]:
def feature_scaling(X):
    '''
    Mean normalized each feature column.

    Parameters
    ----------
    X (np.ndarray) : (m, n) numpy matrix representing feature matrix

    Returns
    -------
        A (m, n) numpy matrix where each column has been mean-normalized.
    '''
    """ YOUR CODE HERE """
    return (X - np.mean(X, axis=0)) / (np.std(X, axis=0))
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [35]:
# Test cases
public_X = np.array([[1, 133], [4, 700], [5, 133], [8, 700]])
expected = np.array([[-1.4, -1], [-0.2, 1], [0.2, -1], [1.4, 1]])

assert np.array_equal(feature_scaling(public_X), expected)

### Task 3.4: Find number of epochs to converge

Fill in the function `find_number_of_epochs(X, y, lr, delta_loss)` that that returns:

* $w_0$ - a number representing the bias constant
* $w_1, w_2, \dots, w_n$ - $(n, 1)$ NumPy matrix, where each element denotes the weight constant of a certain feature
* $num\_of\_epochs$ - a number representing the number of epochs performed to reach convergence

A single epoch is defined as performing the gradient descent *once* and calculating the loss. The loss calculation and gradient descent should be performed using MSE.

The definition of convergence is as follows:

$$ |J_{t-1} - J_{t}| < delta\_loss $$

where $J_{t-1}$ is loss at timestep $t-1$ (previous timestep), $J_{t}$ is loss at timestep $t$ (current timestep), and $delta\_loss$ is the termination criterion.


In [82]:
def find_number_of_epochs(X, y, lr, delta_loss):
    '''
    Do gradient descent until convergence and return number of epochs
    required.

    Parameters
    ----------
    X (np.ndarray) : (m, n) numpy matrix representing feature matrix
    y (np.ndarray) : (m, 1) numpy matrix representing target values
    lr (float) : Learning rate
    delta_loss (float) : Termination criterion
    
    Returns
    -------
        bias (float):
            The bias constant
        weights (np.ndarray):
            A (n, 1) numpy matrix that specifies the weight constants.
        num_of_epochs (int):
            Number of epochs to reach convergence.
        current_loss (float):
            The loss value obtained after convergence.
    '''
    # Do not change
    bias = 0
    weights = np.full((X.shape[1], 1), 0).astype(float)
    num_of_epochs = 0
    previous_loss = np.inf
    # current_loss = -1e14    

    N = X.shape[0]
    pred = X @ weights + bias
    current_loss = mean_squared_error(y, pred)
    while abs(previous_loss - current_loss) >= delta_loss:
        previous_loss = current_loss
        pred = X @ weights + bias
        g_w = -(1 / N) * (X.T @ (y - pred))
        g_b = -(1 / N) * np.sum(y - pred)
        bias -= lr * g_b
        weights -= lr * g_w
        pred = X @ weights + bias
        current_loss = mean_squared_error(y, pred)
        num_of_epochs += 1
    return bias, weights, num_of_epochs, current_loss

In [84]:
# Test cases
poly_X = create_polynomial_matrix(X[:, 2].reshape((-1, 1)), 3)
_, _, num_of_epochs, _ = find_number_of_epochs(poly_X, y, 1e-5, 1e7)
print(find_number_of_epochs(poly_X, y, 1e-5, 1e7))
assert num_of_epochs > 0

(12496.906405098289, array([[21585.28999284],
       [38708.26369539],
       [-4515.94504127]]), 3953, 124453082703.73758)


### Task 3.5: Analyze the effects of feature scaling on Gradient Descent

In this task, we will examine the influence of feature scaling on the efficiency of gradient descent algorithms. Specifically, we utilize a degree 3 polynomial feature matrix derived from a dataset of schools.

<figure>
<img src="./images/loss_vs_epochs.png" alt="loss against epochs with and without normalization" width="100%">
</figure>

The above figure shows plots of loss against epoch count, for gradient descent without and with normalization on the mentioned dataset. Focus on two key aspects:

1. Convergence analysis
    - Observe the number of epochs required to achieve convergence for both the non-normalized (original) and normalized feature matrices.  
    This comparison should be conducted across various learning rates.
2. Loss visualization
    - Analyze the loss values corresponding to both the non-normalized and normalized matrices as a function of epoch count, again considering different learning rates.


**Draw 2 observations or conclusions** from the above figures.


# Problem Set 3 Part B: Credit Card Fraud Detection

## Overview

In class, we discussed about logistic regression, and how it can be useful as a classification algorithm. In Part B of this problem set, we get some hands-on practice by implementing logistic regression on a Credit Card Fraud Detection dataset. Note that for Part B, you should only be using the scikit-learn (sklearn) library for the last part (Task 6.1) on SVM.

**Required Files**:

* ps3.ipynb
* credit_card.csv

**Plagiarism Policy**: Please refer to our [Course Policies](https://canvas.nus.edu.sg/courses/62323/pages/course-policies)

**IMPORTANT**: While it is possible to write and run Python code directly in Jupyter notebook, we recommend that you do this Problem set with an IDE using the .py file provided. An IDE will make debugging significantly easier.

**Post-Problem Set Survey**:
Your feedback is important to us! After completing Problem Set 3, please take a moment to share your thoughts by filling out this [survey](https://coursemology.org/courses/2851/surveys/2410).

## Important

Similar to PS0, your implementation in the following tasks **should NOT involve any iteration, including `map` and `filter`, or recursion**. Instead, you should work with the operations available in NumPy. Solutions that violate this will be penalised.

- You are allowed to use any mathematical functions, but this does not mean that you are allowed to use any NumPy function (there are NumPy functions that aren’t mathematical functions). For example, `np.vectorize` is not allowed since it is iterative. If you are in doubt about which functions are allowed, you should ask in the forum.

There is, however, an exception for **Task 5.4**. In the pseudo-code for the algorithm required, there is an explicit for loop. Hence, only for this task you may use **a single for loop** to iterate for the number of epochs required.

## Introduction

Fraudulent credit card transaction is a common phenomenon in many parts of the world and can lead to potentially large amounts of losses for both companies and customers. Therefore, we hope to help credit card companies recognize those fraudulent transactions so that customers are not charged for items that they did not purchase.

We are given a dataset that contains transactions made by credit cards holders in `credit_card.csv`. If we think about what type of data might be included in the input variables under the given context, we might realize that those input variables are likely to include word descriptions, such as shop name or locations. In this problem set, we don't need to worry about language processing as the data are pre-processed to contain only numeric values.

## Getting Started

Take a look at the columns in the dataset `credit_card.csv`. We have V1-V20, 'Amount', and 'Time' as input features, and 'Class' as output which takes the value 1 if it's fraud and 0 otherwise. This dataset presents 492 frauds out of 284,807 transactions. That means, there are 284,808 rows (including the header) in the csv file.

We will use this dataset to implement logistic regression using stochastic gradient descent for binary classification.

In [ ]:
# Initial imports and setup

import numpy as np
import os
import pandas as pd

from sklearn import svm
from sklearn import model_selection

# Read credit card data into a Pandas dataframe for large tests

dirname = os.getcwd()
credit_card_data_filepath = os.path.join(dirname, 'credit_card.csv')

credit_df = pd.read_csv(credit_card_data_filepath)
X_task5 = credit_df.values[:, :-1]
y_task5 = credit_df.values[:, -1:]

## Basics of Pandas

[Pandas](https://pandas.pydata.org/) is an open source data analysis and manipulation tool in Python. In this problem set, we read the CSV into a [Pandas dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) to provide some nifty methods that makes it easier for us to handle large amounts of data. You can think of a dataframe as a large table that stores our dataset in a neat and optimized manner, making it fast for retrieval and manipulation and data. Using Pandas, we can quickly gain an overview of the type and values of the data stored, distributions of values within the dataset, and even ways to perform sampling.

In the new few sections, we will explore some basic functions of Pandas to help us get started. You do not need to submit any codes for this section, and can simply run the cells to follow along.

### Exploring the dataset

Using the [`head`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html) method on the dataframe, we can get an overview of the data. By default, the method returns the first 5 entries in the dataframe.

In [ ]:
credit_df.head()

Next, we can inspect the [value counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) of the 'Class' property in the dataframe to know the number of fraudulent and non-fraudulent transactions.

In [ ]:
# Inspect the number of fraudulent and non-fraudulent transactions.
credit_df['Class'].value_counts()

### Indexing and selecting data

Similar to NumPy, we can also index and select data on Pandas dataframes.

For example, we can [select columns in the dataframe by their labels](https://pandas.pydata.org/docs/user_guide/indexing.html#basics). In the following example, we use `'Class'` to index the 'Class' column in the credit dataframe.

In [ ]:
# Select the 'Class' column in the credit dataframe
credit_df['Class']

Or, we can use integer indexing.

In [ ]:
# Obtain the first 2 rows
credit_df[0:2]

We can also [select columns in the dataframe that fulfils some condition](https://pandas.pydata.org/docs/user_guide/indexing.html#boolean-indexing). In the example below, `credit_df['Class'] == 0` returns a [Pandas Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html) of length 284807, which is the size of our dataset. It contains the value `True` if and only if the `Class` value for the particular entry is of value 0, and `False` otherwise. We can use this Boolean series to index the credit dataframe to return the rows where the `Class` field is 0. Does this remind you of how NumPy arrays operate?

In [ ]:
credit_df['Class'] == 0

In [ ]:
# Obtain the credit dataframe where the 'Class' field is 0
credit_df[credit_df['Class'] == 0]

You can also [concatenate](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) Pandas series or dataframes together! In this example, we explored how we can concat the first 2 rows and last 2 rows of the dataframe. Similar to NumPy, you would also need to specify the axis for concatenation.

In [ ]:
pd.concat([credit_df[:2], credit_df[-2:]], axis=0)

## Data Processing

As you might have realised, managing Pandas is not so much different from how you would operate on NumPy or basic Python structures! In practice, Pandas can be very useful for data resampling (e.g. the [`sample` method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html)).

### Task 4.1: Problem with imbalanced data

Describe in one or two sentences what problem we might encounter if we directly use the given dataset without processing it.

Hint: consider the 'Class' column, and think about how a high accuracy prediction can be achieved in a wrong way


### Concept 4.2: Resampling methods

When we are faced with the issue of imbalanced data, there are several ways to deal with it. A more direct way might be just to collect more data instances. We realized that in our case this doesn't work well because the events unevenly occur. We then look at how to resample the existing instances.

In this problem set, you are introduced with three resampling methods: undersampling, oversampling, and SMOTE.

#### Concept 4.2.1: Undersampling

<figure>
<img src="./images/undersampling.png" alt="visualisation of undersampling" width="50%">
<figcaption style="text-align:center">Figure 6: Visualisation of undersampling.</figcaption>
</figure>

The figure above illustrates undersampling. In undersampling, we remove samples from the majority class. More specifically, we randomly take subsamples of the majority class such that the size of two classes is the same. There is, however, a drawback of undersampling - by removing data randomly, you might have removed some valuable information from the dataset. Is there any way to do better without losing the valuable information?

#### Concept 4.2.2: Oversampling

<figure>
<img src="./images/oversampling.png" alt="visualisation of oversampling" width="50%">
<figcaption style="text-align:center">Figure 7: Visualisation of oversampling.</figcaption>
</figure>

The figure above illustrates oversampling. In oversampling, we duplicate records of the minority class such that the size of two classes balance out. There is a bright side of oversampling - you don't lose certain valuable information. At the same time, you also realize the drawback of oversampling - it can cause overfitting and a poor generalization of the test set. Now the question is, instead of simply duplicating records, is there any other way to increase the number of records in the minority set?

#### Concept 4.2.3: SMOTE (for further reading only)

<figure>
<img src="./images/smote.png" alt="visualisation of SMOTE" width="50%">
<figcaption style="text-align:center">Figure 8: Visualisation of SMOTE.</figcaption>
</figure>

The figure above illustrates Synthetic Minority Oversampling Technique (SMOTE).

The SMOTE algorithm works in these four steps:

1. Consider minority and majority instances in vector space.
1. For each minority-class instance pair, interpolate their feature values.
1. Randomly synthesize instances and label with minority class
1. More instances added to minority class

In this case, you increase the number of minority instances without simply duplicating the values. Now your newly inserted minority record is not an exact copy of an existing data point, but it is also not too different from the known observations in your minority class. Outside this problem set, when we want to do data resampling, we can use the [Python imbalanced-learn library](https://pypi.org/project/imbalanced-learn/).

#### Word of caution when resampling

In the section above, we introduced three resampling techniques. One very important thing to note is, in practice you should first split dataset to train–test sets, then resample the train dataset before training the model. This is done to avoid data leakage (snooping).

Data leakage can cause you to create overly optimistic if not completely invalid predictive models. It is when information from outside the training dataset is used to train the model. The additional information might allow the model to learn something that it otherwise would not know and in turn invalidate the estimated performance of the mode being constructed.

## Splitting data

Before we can start training our models, we need to **randomly** partition our dataset into training data and testing data. 

### Concept: Splitting data into train and test sets

Remember that we are trying to make a model that can predict fraudulence of data points that the model has never seen. It would be unwise to measure the accuracy of the model using the same data it trained on, as that does not give you an idea of how well the model will work on unseen data. Hence, we will have to train the model on a training set and evaluate its effectiveness (e.g. using metrics such as accuracy, F1 score etc.) on a separate test set. The [scikit-learn](https://scikit-learn.org/stable/modules/svm.html) library provides methods to perform a [train-test data split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

Correspondingly, if you intend to do more with your model, for example, fine-tune it by adjust various hyperparameters or compare it with other models, you are advised to use separate development or validation sets respectively (which are disjoint from the training and test sets).

## Logistic regression

### Gradient descent

Recall the gradient descent method you learned and applied in the previous problem set. Gradient descent is an iterative optimization method which finds the minimum of a differentiable function. It's often used to find the coefficients that minimize the cost function. Here is a brief recap of how that's done:

<figure>
<img src="./images/gradient_descent.png" alt="visualisation of gradient descent" width="50%">
    <figcaption style="text-align:center"><a href="https://medium.com/@divakar_239/stochastic-vs-batch-gradient-descent-8820568eada1">Figure 9: Visualisation of gradient descent.</a></figcaption>
</figure>

We start at an initial weight vector $\mathbf{w} = (w_0, w_1, ..., w_n)$. Then we take incremental steps along the steepest slope to get to the bottom where the minimum cost lies. Along the way, we keep updating the weight coefficients by computing the gradients using the training samples from the training dataset.

### Concept: Batch gradient descent

In batch gradient descent, **all** the training data (entire batch) is taken into consideration in a single step. For one step of gradient descent, we calculate $w_j \leftarrow w_j - \alpha \frac{\partial J(\mathbf{w})}{\partial w_j}$ (where $\alpha$ is the learning rate) simultaneously for all $j$ based on all the training examples and then use that mean gradient to update our parameters.

By continuing this for iterations until convergence, or until it's time to stop, we reach the optimal or near optimal parameters $\mathbf{w}$ leading to minimum cost.

### Concept: Stochastic gradient descent

Think about what's a limitation of the (entire) batch gradient descent introduced earlier. What if the dataset is very large? In Stochastic Gradient Descent (SGD), we consider only the error on a single record $(x^{(*)}, y^{(*)})$ at a time. How does this work?

During the training loop, we will randomly sample an instance in the dataset. This instance will be used to update the weights of the model $w_j \leftarrow w_j - \alpha \frac{\partial J_{stoch}(\mathbf{w})}{\partial w_j}$ (where $\alpha$ is the learning rate) simultaneously for all $j$.

Note that SGD does not necessarily decrease the batch loss in each iteration! However, on average, the loss will still decrease. Since we work on only a single record at any time, SGD provides the advantage of a much smaller memory requirement, and faster computation time. SGD also comes with its own set of disadvantages, for example, we lose benefits of any vectorization.

### Concept: Gradient descent termination condition

The next question is when to stop? The ideal case is to run until convergence, but convergence might be hard to obtain. Here are some criterias you can use:

* Stop when error change is small and/or
* Stop when error is small
* Stop when maximum number of iterations is reached

Now let's try to apply what we learned about batch gradient descent and stochastic gradient descent.
Task 5.1 to 5.5 will gradually guide you to complete the logistic regression using stochastic gradient descent. For the task in this problem set, you can assume that the bias column has been added for all input `X`. That means given `X` as an argument, you don't need to manually add the bias column again in your code.

### Task 5.1: Cost function

Recall that for logistic regression, we want an error function for an individual value to be:

<figure>
<img src="./images/piecewise_error.png" alt="error function" width="50%">
<figcaption style="text-align:center">Figure 10: Cost function.</figcaption>
</figure>

We can simplify the condition and transform it into the equivalent cost function:

<figure>
<img src="./images/error.png" alt="error function as a line" width="50%">
<figcaption style="text-align:center">Figure 11: Cost without conditions.</figcaption>
</figure>

In this task, you need to implement `cost_function` $E$ as mentioned above. This function takes `X`, `y`, and `weight_vector` $\mathbf{w}$ as arguments, and returns the error $E$. Note that for this task, the $E$ should account for **all** the training data.

Here, we are using the $\log$ function and we need to handle the case of computing $\log(0)$. There are many ways to handle this. In this task, we will handle $\log(0)$ by using the machine epsilon for numpy `float64` type, and use the trick $\log(x + eps)$ which allows $x$ to be $0$. If $x$ was any other value, $\log(x + eps)$ would be very close to $\log(x)$. This helps to improve numerical stability in computation.


In [ ]:
def cost_function(X: np.ndarray, y: np.ndarray, weight_vector: np.ndarray):
    '''
    Cross entropy error for logistic regression

    Parameters
    ----------
    X: np.ndarray
        (m, n) training dataset (features).
    y: np.ndarray
        (m,) training dataset (corresponding targets).
    weight_vector: np.ndarray
        (n,) weight parameters.

    Returns
    -------
    Cost
    '''
    
    # Machine epsilon for numpy `float64` type
    eps = np.finfo(np.float64).eps
    
    """ YOUR CODE HERE """
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [ ]:
# Test cases
data1 = [[111.1, 10, 0], [111.2, 20, 0], [111.3, 10, 0], [111.4, 10, 0], [111.5, 10, 0], [111.6, 10, 1], [111.4, 10, 0], [111.5, 10, 1], [111.6, 10, 1]]
df1 = pd.DataFrame(data1, columns = ['V1', 'V2', 'Class'])
X1 = df1.iloc[:, :-1].to_numpy()
y1 = df1.iloc[:, -1].to_numpy()
w1 = np.transpose([0.002, 0.1220])

assert np.round(cost_function(X1, y1, w1), 5) == np.round(1.29333, 5)

### Task 5.2: Weight update

In this task, you need to implement `weight_update`. This function takes `X`, `y`, `alpha`, and a `weight_vector` as arguments, and output the new weight vector. Each call to the function should make one update on the weight vector.


In [ ]:
def weight_update(X: np.ndarray, y: np.ndarray, alpha: np.float64, weight_vector: np.ndarray) -> np.ndarray:
    '''
    Do the weight update for one step in gradient descent

    Parameters
    ----------
    X: np.ndarray
        (m, n) training dataset (features).
    y: np.ndarray
        (m,) training dataset (corresponding targets).
    alpha: np.float64
        logistic regression learning rate.1
    weight_vector: np.ndarray
        (n,) weight parameters.

    Returns
    -------
    New weight vector after one round of update.
    '''

    """ YOUR CODE HERE """
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [ ]:
# Test cases
data1 = [[111.1, 10, 0], [111.2, 20, 0], [111.3, 10, 0], [111.4, 10, 0], [111.5, 10, 0], [111.6, 10, 1],[111.4, 10, 0], [111.5, 10, 1], [111.6, 10, 1]]
df1 = pd.DataFrame(data1, columns = ['V1', 'V2', 'Class'])
X1 = df1.iloc[:, :-1].to_numpy()
y1 = df1.iloc[:, -1].to_numpy()
w1 = np.transpose([2.2000, 12.20000])
a1 = 1e-5
nw1 = np.array([2.199,12.2])

assert np.array_equal(np.round(weight_update(X1, y1, a1, w1), 3), nw1)

### Task 5.3: Logistic regression classification

Remember that logistic regression is used for classification even though the function gives a probability output. In this task, you classify each element in `X`, given `weight_vector` using `prob_threshold` as the threshold, and output the classification result as an `np.ndarray`.

If the probability predicted by the `weight_vector` exceeds the `prob_threshold`, we should classify it as fraud (`y = 1`). Otherwise, we should classify it as legitimate (`y = 0`).


In [ ]:
def logistic_regression_classification(X: np.ndarray, weight_vector: np.ndarray, prob_threshold: np.float64=0.5):
    '''
    Do classification task using logistic regression.

    Parameters
    ----------
    X: np.ndarray
        (m, n) training dataset (features).
    weight_vector: np.ndarray
        (n,) weight parameters.
    prob_threshold: np.float64
        the threshold for a prediction to be considered fraudulent.

    Returns
    -------
    Classification result as an (m,) np.ndarray
    '''

    """ YOUR CODE HERE """
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [ ]:
# Test cases
data1 = [[111.1, 10, 0], [111.2, 20, 0], [111.3, 10, 0], [111.4, 10, 0], [111.5, 10, 0], [211.6, 80, 1],[111.4, 10, 0], [111.5, 80, 1], [211.6, 80, 1]]
df1 = pd.DataFrame(data1, columns = ['V1', 'V2', 'Class'])
X1 = df1.iloc[:, :-1].to_numpy()
y1 = df1.iloc[:, -1].to_numpy()
w1 = np.transpose([-0.000002, 0.000003])
expected1 = np.transpose([0, 0, 0, 0, 0, 0, 0, 1, 0])
result1 = logistic_regression_classification(X1, w1)

assert result1.shape == expected1.shape and (result1 == expected1).all()

### Task 5.4: Logistic regression using stochastic gradient descent

In this task, you need to implement logistic regression using stochastic gradient descent. This function takes `X_train`, `y_train`, `max_num_iterations`, `threshold`, `alpha`, and `seed` as arguments, and output the final `weight_vector` you obtained. You can make use of the previous functions you have implemented earlier.

For stochastic gradient descent, an epoch refers to one pass through **one randomly selected data point**. Hence, one iteration of gradient descent, which updates the weight vector based on the cost function computed on the randomly selected data point, corresponds to one epoch. Stochastic gradient descent is terminated when the number of update rounds exceeds the maximum number of epochs (`max_num_iterations`) given, or when the computed error decreases to or below the threshold value (`threshold`).

For this task, you should use `np.random.choice` to randomly choose the training data for each iteration. You should also use the `seed` argument with `np.random.seed(seed)` to set the random seed for reproducibility.

For your convenience, you may define your own `weight_update_stochastic` method, which does the weight update for one step in stochastic gradient descent, inside the `logistic_regression_stochastic_gradient_descent` method.


In [ ]:
def logistic_regression_stochastic_gradient_descent(X_train: np.ndarray, y_train: np.ndarray, max_num_iterations: int=250, threshold: np.float64=0.05, alpha: np.float64=1e-5, seed: int=43) -> np.ndarray:
    '''
    Initialize your weight to zeros. Write a terminating condition, and run the weight update for some iterations.
    Get the resulting weight vector.

    Parameters
    ----------
    X_train: np.ndarray
        (m, n) training dataset (features).
    y_train: np.ndarray
        (m,) training dataset (corresponding targets).
    max_num_iterations: int
        this should be one of the terminating conditions. 
        The gradient descent step should happen at most max_num_iterations times.
    threshold: np.float64
        terminating when error <= threshold value, or if you reach the max number of update rounds first.
    alpha: np.float64
        logistic regression learning rate.
    seed: int
        seed for random number generation.

    Returns
    -------
    The final (n,) weight parameters
    '''

    """ YOUR CODE HERE """
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [ ]:
# Test cases
data1 = [[111.1, 10, 0], [111.2, 20, 0], [111.3, 10, 0], [111.4, 10, 0], [111.5, 10, 0], [211.6, 80, 1],[111.4, 10, 0], [111.5, 80, 1], [211.6, 80, 1]]
df1 = pd.DataFrame(data1, columns = ['V1', 'V2', 'Class'])
X1 = df1.iloc[:, :-1].to_numpy()
y1 = df1.iloc[:, -1].to_numpy()
expected1 = cost_function(X1, y1, np.transpose(np.zeros(X1.shape[1])))

assert cost_function(X1, y1, logistic_regression_stochastic_gradient_descent(X1, y1)) < expected1

### Task 5.5: Stochastic gradient descent vs batch gradient descent

Recall that it is ideal for gradient descent to run until convergence, but convergence might be hard to obtain.

In this task, we will explore the relationship between the cross entropy loss vs the number of update rounds and/or the runtime it takes to run the update for both batch gradient descent and stochastic gradient descent using a sample of the `credit_card` dataset.

Upon measuring the runtime of batch and stochastic gradient descent for different epoch counts, we get the following graphs:

<figure>
<img src="./images/batch_vs_stochastic_update_rounds.png" alt="CE loss against epoch count" width="50%">
<figcaption style="text-align:center">Figure 12: cross entropy loss vs the number of update rounds for batch and stochastic gradient descent.</figcaption>
</figure>

<figure>
<img src="./images/batch_vs_stochastic_runtime.png" alt="CE loss against runtime" width="50%">
<figcaption style="text-align:center">Figure 13: cross entropy loss vs runtime for batch and stochastic gradient descent.</figcaption>
</figure>

Do note that the runtime is the cumulative runtime of the entire run of gradient descent, not the runtime per individual update round. 

With reference to figures 12 and 13, what do you observe about the relationship between the cross entropy loss vs the number of update rounds and/or the runtime it takes to run the update for both batch gradient descent and stochastic gradient descent?

Hint: You may/may not angle your answer in terms of explaining the effect of the size of the dataset, whether the algorithm will be stuck in a local minima and so on.


## Support Vector Machine

Now let's apply the support vector machine we learnt in lecture to the credit card dataset. Here is a quick recap of what SVM is about: We are given a training dataset of $n$ points of the form $(x^{(i)}, y^{(i)})$, where the $y^{(i)}$ are labels, each indicating the class to which the point $x^{(i)}$ belongs. Typically, the SVM is formulated with $y^{(i)}$ being either 1 or -1, however the following discussion uses equivalently $y^{(i)}$ being either 1 or 0. Be careful that some examples in this problem use data with 1,0 and some examples use data with 1,-1. 

Each $x^{(i)}$ $n$-dimensional real vector. We want to find the "maximum-margin hyperplane" that divides the group of points $x^{(i)}$ for which $y_{i}=1$ from the group of points for which $y_{i}=0$, which is defined so that the distance between the hyperplane and the nearest point $x^{(i)}$ from either group is maximized.

<figure>
<img src="./images/svm.png" alt="visualisation of support vector machine" width="50%">
<figcaption style="text-align:center">Figure 14: Visualisation of support vector machine.</figcaption>
</figure>

How can we do construct such a "maximum-margin hyperplane"? Recall that in the lecture, we formulated soft-margin SVMs as the following minimization problem:

$$
\min_{w} C \left[\sum_{i = 1}^{m} y^{(i)}\ Cost_1(\mathbf{w}^{T} \mathbf{x}^{(i)}) + (1 - y^{(i)})\ Cost_0(\mathbf{w}^{T} \mathbf{x}^{(i)})\right] + \frac{1}{2} \sum_{i = 1}^{n} \mathbf{w}_i^2 $$

where $Cost_1(\mathbf{w}^T \mathbf{x}^{(i)}) = \max(0, 1 - \mathbf{w}^T \mathbf{x}^{(i)})$ and $Cost_0(\mathbf{w}^T \mathbf{x}^{(i)}) = \max(0, 1 + \mathbf{w}^T \mathbf{x}^{(i)})$, with the hypothesis function $h_\mathbf{w}(\mathbf{w}) = 1$ if $\mathbf{w}^T \mathbf{x} \geq 0$, and 0 otherwise.

In particular, the summand term is known as the *hinge loss*. Thus, when $y^{(i)} = 0$, the more negative $\mathbf{w}^T \mathbf{x}$ is, the more $Cost_1$ penalises the model. Similarly, $Cost_0$ penalises the model the further $\mathbf{w}^T \mathbf{x}$ strays from the negative region. Notice how the cost function aligns with the goal of the hypothesis function? Additional note: $Cost_0 = Cost_{-1}$, where $Cost_{-1}$ was used in the lecture. 

Furthermore, we also add a regularization parameter $C$ to the cost function to balance the trade-off between margin maximization and loss.

Thus, solving this minimization problem allows us to find the weight vector $\mathbf{w}$ that provides the maximum-margin hyperplane while balancing the trade-off of overfitting.

With this set up in mind, how can we implement this in code? Thankfully, with modern machine learning libraries, much of the tedious work of solving this minimization problem has been done. As machine learning practitioners, one popular library that we commonly use (aside from [NumPy](https://numpy.org/doc/stable/index.html)) is [scikit-learn](https://scikit-learn.org/), which is built on top of NumPy and provides simple and efficient tools to implement common machine learning algorithms.

For the following task, we will focus on comparing Linear and Gaussian Kernel SVMs.

### Task 6.1: Linear SVM vs Gaussian Kernel SVM

In this task, the code for `linear_svm` and `gaussian_kernel_svm` has been given to you. These functions take in `X` and `y` (the entire dataset), do a train test data split with `test_size` of 0.3 and `random_state` of 42, create an instance of a Linear/Gaussian kernel SVM classifier using the default parameters, train the classifier, and output the predictions as well as the accuracy score. Run the code cells below (you may modify them if you wish).

Based on your observations, when using support vector machines, how do you think we should choose between linear kernel vs Gaussian kernel? In other words, which kernel is better in what cases? Give 2 reasons why one might be more suitable than the other in different cases.


In [ ]:
def linear_svm(X: np.ndarray, y: np.ndarray):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.3, random_state=42)

    clf = svm.SVC(kernel='linear')
    clf.fit(X_train, y_train)

    clf_predictions = clf.predict(X_test)

    return clf_predictions, clf.score(X_test, y_test) * 100


# small data
# Do note that y values for data1 are either 0 or 1 for this half of the task, but typically are 
#   either -1 or 1 for SVMs. You do not have to change this data for this task.
data1 = [[111.1, 10, 0], [111.2, 20, 0], [111.3, 10, 0], [111.4, 10, 0], [111.5, 10, 0], [211.6, 80, 1],
        [111.4, 10, 0], [111.5, 80, 1], [211.6, 80, 1]]
df1 = pd.DataFrame(data1, columns = ['V1', 'V2', 'Class'])
X1 = df1.iloc[:, :-1].to_numpy()
y1 = df1.iloc[:, -1].to_numpy()
result1 = linear_svm(X1, y1)

# subset of credit card data
class_0 = credit_df[credit_df['Class'] == 0]
class_1 = credit_df[credit_df['Class'] == 1]

data_0 = class_0.sample(n=15, random_state=42)
data_1 = class_1.sample(n=50, random_state=42)
data_100 = pd.concat([data_1, data_0], axis=0)
X_task6 = data_100.iloc[:, :-1].to_numpy()
y_task6 = data_100.iloc[:, -1].to_numpy()

result = linear_svm(X_task6, y_task6.ravel())

In [ ]:
def gaussian_kernel_svm(X: np.ndarray, y: np.ndarray):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.3, random_state=42)

    gaussian_kernel_classifier = svm.SVC(kernel='rbf')
    gaussian_kernel_classifier.fit(X_train, y_train)

    gaussian_kernel_classifier_predictions = gaussian_kernel_classifier.predict(X_test)

    return gaussian_kernel_classifier_predictions, gaussian_kernel_classifier.score(X_test, y_test) * 100


# small data
data1 = [[111.1, 10, -1], [111.2, 20, -1], [111.3, 10, -1], [111.4, 10, -1], [111.5, 10, -1], [211.6, 80, 1],
        [111.4, 10, -1], [111.5, 80, 1], [211.6, 80, 1]]
df1 = pd.DataFrame(data1, columns = ['V1', 'V2', 'Class'])
X1 = df1.iloc[:, :-1].to_numpy()
y1 = df1.iloc[:, -1].to_numpy()
result1 = gaussian_kernel_svm(X1, y1)

# subset of credit card data
class_0 = credit_df[credit_df['Class'] == 0]
class_1 = credit_df[credit_df['Class'] == 1]

data_0 = class_0.sample(n=15, random_state=42)
data_1 = class_1.sample(n=50, random_state=42)
data_100 = pd.concat([data_1, data_0], axis=0)
X_task6 = data_100.iloc[:, :-1].to_numpy()
y_task6 = data_100.iloc[:, -1].to_numpy()

result = gaussian_kernel_svm(X_task6, y_task6.ravel())

## Submission

Once you are done, please submit your work to Coursemology, by copying the right snippets of code into the corresponding box that says "Your answer,"and click "Save." After you save, you can still make changes to your submission.

Once you are satisfied with what you have uploaded, click "Finalize submission." Note that once your submission is finalized, it is considered to be submitted for grading and cannot be changed. If you need to undo this action, you will have to email your assigned tutor for help. Please do not finalize your submission until you are sure that you want to submit your solutions for grading.

*Have fun and enjoy coding.*